In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def is_equivalent_transform(euler1, t1, euler2, t2, atol=1e-6):
    """
    判断两组 [欧拉角 + 平移向量] 是否表示同一个外参变换

    参数:
    - euler1, euler2: shape=(3,), 欧拉角，ZYX顺序，单位：弧度
    - t1, t2: shape=(3,), 平移向量
    - atol: 容差（默认1e-6）

    返回:
    - bool: True 表示等价，False 表示不同
    """

    # ZYX 顺序：先绕 X，后绕 Y，再绕 Z
    rot1 = R.from_euler('zyx', euler1)
    rot2 = R.from_euler('zyx', euler2)

    R1 = rot1.as_matrix()
    R2 = rot2.as_matrix()

    # 比较旋转和平移是否一致（考虑浮点误差）
    same_rotation = np.allclose(R1, R2, atol=atol)
    same_translation = np.allclose(t1, t2, atol=atol)

    return same_rotation and same_translation


In [6]:
def is_equivalent_transform_deg(euler_deg1, t1, euler_deg2, t2, atol_deg=5, atol_trans=0.3):
    """
    判断两组 [欧拉角(角度) + 平移向量] 是否表示同一个外参变换

    参数:
    - euler_deg1, euler_deg2: shape=(3,), 欧拉角（ZYX顺序，单位：度）
    - t1, t2: shape=(3,), 平移向量
    - atol_deg: 角度容差（默认 1e-3 度）
    - atol_trans: 平移容差（默认 1e-6 米）

    返回:
    - bool: True 表示等价，False 表示不同
    """

    # 构造旋转矩阵（从角度）
    rot1 = R.from_euler('xyz', euler_deg1, degrees=True)
    rot2 = R.from_euler('xyz', euler_deg2, degrees=True)

    R1 = rot1.as_matrix()
    R2 = rot2.as_matrix()

    # 比较旋转矩阵是否一致（使用角度容差近似）
    same_rotation = np.allclose(R1, R2, atol=np.sin(np.deg2rad(atol_deg)))
    same_translation = np.allclose(t1, t2, atol=atol_trans)

    return same_rotation and same_translation

In [9]:
euler_deg1 = np.array([69.41797658585301, 179.8261575509262, 89.94677790440619])
t1 = np.array([0.4672740459251404, -0.077, 0.91])

euler_deg2 = np.array([13.074538, 106.868965, -102.851486])  # 370° 与 10° 等价（多圈）
t2 = np.array([ 0.228194, 1.865215, -1.795097])

print(is_equivalent_transform_deg(euler_deg1, t1, euler_deg2, t2,100,3))  # 输出 True


False


In [14]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def print_rt_decomposition(rt_matrix: np.ndarray):
    """
    输出一个 4x4 RT 矩阵在多个旋转顺序下的欧拉角（角度）和平移向量。
    """
    assert rt_matrix.shape == (4, 4), "输入必须是 4x4 的变换矩阵"

    rotation_matrix = rt_matrix[:3, :3]
    translation = rt_matrix[:3, 3]
    r = R.from_matrix(rotation_matrix)

    orders = ['xyz', 'zyx', 'yxz', 'zxy', 'yzx', 'xzy']

    print(f"\n=== Translation Vector ===\n[x, y, z] = {translation}\n")

    for order in orders:
        try:
            euler_deg = r.as_euler(order, degrees=True)
            print(f"--- Rotation Order: {order.upper()} ---")
            print(f"Euler angles (deg) [X, Y, Z] = {euler_deg}\n")
        except ValueError:
            print(f"--- Rotation Order: {order.upper()} ---")
            print("Euler angles not defined due to singularity.\n")


In [15]:
rt = np.array([
    [0.064544, -0.282915 ,0.956971 ,0.402355],
    [-0.997827 ,-0.005597 ,0.065645 ,-0.072086],
    [-0.013216 ,-0.959129 ,-0.282662, 0.877380],
    [0,  0, 0, 1]
])

print_rt_decomposition(rt)



=== Translation Vector ===
[x, y, z] = [ 0.402355 -0.072086  0.87738 ]

--- Rotation Order: XYZ ---
Euler angles (deg) [X, Y, Z] = [-106.42060243    0.7572369   -86.29900318]

--- Rotation Order: ZYX ---
Euler angles (deg) [X, Y, Z] = [  77.14853579   73.13101238 -166.92546806]

--- Rotation Order: YXZ ---
Euler angles (deg) [X, Y, Z] = [177.32307627 -73.562427    91.13333313]

--- Rotation Order: ZXY ---
Euler angles (deg) [X, Y, Z] = [-90.32137379  -3.76388828 106.4556277 ]

--- Rotation Order: YZX ---
Euler angles (deg) [X, Y, Z] = [ 86.14146179  16.43426619 -90.33434022]

--- Rotation Order: XZY ---
Euler angles (deg) [X, Y, Z] = [-94.87326118 -86.2224363   11.57180682]



In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np

euler_deg1 = [30, 45, 60]  # XYZ 顺序，单位：度
rot1 = R.from_euler('xyz', euler_deg1, degrees=True)
rotation_matrix = rot1.as_matrix()  # 得到3x3旋转矩阵

print(rotation_matrix)

In [16]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def euler_to_rt_matrix(euler_deg, translation, order):
    rot = R.from_euler(order, euler_deg, degrees=True)
    rt = np.eye(4)
    rt[:3, :3] = rot.as_matrix()
    rt[:3, 3] = translation
    return rt

def compare_rt_with_all_orders(euler_deg, translation, true_rt, atol=1e-6):
    """
    将给定的欧拉角和位移，在多个顺序下转为外参矩阵，与 true_rt 比较。

    参数：
    - euler_deg: 欧拉角 [rx, ry, rz]，单位为度
    - translation: 平移向量 [tx, ty, tz]
    - true_rt: 真实的 4x4 外参矩阵
    - atol: 判断矩阵是否相等的容差

    返回：
    - 匹配的顺序列表
    """
    orders = ['xyz', 'zyx', 'yxz', 'zxy', 'xzy', 'yzx']
    matches = []
    for order in orders:
        est_rt = euler_to_rt_matrix(euler_deg, translation, order)
        if np.allclose(est_rt, true_rt, atol=atol):
            matches.append(order)
            print(f"✅ 匹配顺序: {order}")
        else:
            print(f"❌ 不匹配顺序: {order}")
    return matches


In [23]:
# 给定的欧拉角（单位：度）和位移
euler_deg = [69.41797658585301, 179.8261575509262, 89.94677790440619]
translation = [0.4672740459251404, -0.077, 0.91]

# 构造一个“真实外参矩阵”（比如来自标定结果）
true_rt = np.array([
    [0.064544, -0.282915 ,0.956971 ,0.402355],
    [-0.997827 ,-0.005597 ,0.065645 ,-0.072086],
    [-0.013216 ,-0.959129 ,-0.282662, 0.877380],
    [0,  0, 0, 1]
])
# 进行比较
matches = compare_rt_with_all_orders(euler_deg, translation, true_rt,1e-1)
print("\n可能正确的旋转顺序：", matches)


✅ 匹配顺序: xyz
❌ 不匹配顺序: zyx
❌ 不匹配顺序: yxz
❌ 不匹配顺序: zxy
❌ 不匹配顺序: xzy
❌ 不匹配顺序: yzx

可能正确的旋转顺序： ['xyz']
